# Loading tables from the Tascomi API

This notebook allows the user to test the Tascomi API endpoint for a specified table, and view output data within a Pandas dataframe. This notebook is intended for local use.

## Import required modules

In [ ]:
import base64
from datetime import date, datetime, timedelta
from dateutil import tz
import hmac, hashlib
import pandas as pd
import requests
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Set variables

In [ ]:
PUBLIC_KEY = ""
PRIVATE_KEY = ""
public_key = PUBLIC_KEY.encode('utf-8')
private_key = PRIVATE_KEY.encode('utf-8')

table_to_read = ""
request_uri = f'https://hackney-planning.tascomi.com/rest/v1/{table_to_read}'
request_method = 'GET'

## Define class for API request

In [ ]:
# Python 2 code provided by supplier so amended to work in Python 3
# Also ensured that the current time is set to local (London) timeszone so that BST accounted for
class APIClass:
    def __init__(self, public_key, private_key, request_uri, request_method, data=None, debug=False):
        tz_ldn = tz.gettz('Europe/London')
        now = datetime.now(tz_ldn)
        the_time = now.strftime("%Y%m%d%H%M").encode('utf-8')
        print(f'The time: {the_time}\n')
        crypt = hashlib.sha256(public_key + the_time)
        token = crypt.hexdigest().encode('utf-8')
        hash = base64.b64encode(hmac.new(private_key, token, hashlib.sha256).hexdigest().encode('utf-8'))
        self.token = token
        self.public_key = public_key
        self.private_key = private_key
        self.request_uri = request_uri
        self.request_method = request_method
        self.hash = hash
        self.debug = debug
        self.data = data


    def sendRequest(self):
        headers = {'X-Public': self.public_key, 'X-Hash': self.hash, 'X-Debug': str(self.debug)}
        payload = self.data

        if self.request_method == 'GET':
            r = requests.get(self.request_uri, params=payload, headers=headers)
        elif self.request_method == 'POST':
            r = requests.post(self.request_uri, params=payload, headers=headers)
        elif self.request_method == 'PUT':
            r = requests.put(self.request_uri, params=payload, headers=headers)
        elif self.request_method == 'DELETE':
            r = requests.delete(self.request_uri, params=payload, headers=headers)
        
        if r.status_code == 200:
            print(f'Status code less than 400: {r.ok}, Status code: {r.status_code}\n\nHeaders: {r.headers}')
            # return JSON
            return r.json(), r.headers
        else: 
            print(f'Status code: {r.status_code} - problem with request.')


## Send API request for specified Tascomi table

In [ ]:
newClass = APIClass(public_key, private_key, request_uri, request_method, data=None, debug=True)
data, headers = newClass.sendRequest()

num_results = headers.get('X-Number-Of-Results')
page_num = headers.get('X-Page-Number')
results_per_page = headers.get('X-Results-Per-Page')

# Convert JSON to dataframe
df = pd.DataFrame.from_dict(data)

print(f'\n\nNumber results: {num_results}\nPage number: {page_num}\nResults per page: {results_per_page}\n')
df

## END